# Ataxx
- 아래 있는 셀을 바로 실행하시면 됩니다.

In [ ]:
import inspect
from enum import Enum


class Utils:
    @staticmethod
    def copy_board(board):
        return [[*row] for row in board]


class Cell(Enum):
    PLAYER_A = 1
    PLAYER_B = 2
    EMPTY = 0

    def get_opposite(self):
        if self == Cell.PLAYER_A:
            return Cell.PLAYER_B
        elif self == Cell.PLAYER_B:
            return Cell.PLAYER_A

    def get_mark(self):
        if self == Cell.PLAYER_A:
            return 'A'
        elif self == Cell.PLAYER_B:
            return 'B'
        elif self == Cell.EMPTY:
            return ' '


class CoordinateObject:
    __INVALID__ = -999999
    row = __INVALID__
    col = __INVALID__
    delta_row = __INVALID__
    delta_col = __INVALID__

    def is_two_step(self):
        return abs(self.delta_row) == 2 or abs(self.delta_col) == 2

    def is_row_and_col_not_defined(self):
        return self.row == self.__INVALID__ and self.col == self.__INVALID__

    def is_delta_row_and_delta_col_not_defined(self):
        return self.delta_row == self.__INVALID__ and self.delta_col == self.__INVALID__

    def next_row(self):
        return self.row + self.delta_row

    def next_col(self):
        return self.col + self.delta_col

    def is_valid(self):
        return not self.is_row_and_col_not_defined() and not self.is_delta_row_and_delta_col_not_defined()


class AtaxxService:
    def __init__(self, n, level):
        self.N = n
        self.level = level

    def is_valid_range(self, row, col):
        return 0 <= row < self.N and 0 <= col < self.N

    def heuristic(self, board, cell):
        return self.__heuristic_impl(board, cell, self.level)

    def apply_change_to_board(self, board, obj, cell):
        if obj.is_two_step():
            board[obj.row][obj.col] = Cell.EMPTY
        board[obj.next_row()][obj.next_col()] = cell
        self.__change_colors(board, obj.next_row(), obj.next_col(), cell)

    def __change_colors(self, board, row, col, cell):
        dx = [-1, 0, 1, 1, 1, 0, -1, -1]
        dy = [1, 1, 1, 0, -1, -1, -1, 0]

        for i in range(8):
            target_col = col + dx[i]
            target_row = row + dy[i]

            if not self.is_valid_range(target_row, target_col):
                continue

            if board[target_row][target_col] == cell.get_opposite():
                board[target_row][target_col] = cell

    def __heuristic_impl(self, board, cell, level):
        temp = Utils.copy_board(board)

        # 한 칸 짜리
        dx_1 = [-1, 0, 1, 1, 1, 0, -1, -1]
        dy_1 = [1, 1, 1, 0, -1, -1, -1, 0]

        # 두 칸 짜리
        dx_2 = [-2, -1, 0, 1, 2, 2, 2, 2, 2, 1, 0, -1, -2, -2, -2, -2]
        dy_2 = [-2, -2, -2, -2, -2, -1, 0, 1, 2, 2, 2, 2, 2, 1, 0, -1, -2]

        dx_list = dx_1 + dx_2
        dy_list = dy_1 + dy_2

        max_diff = -999_999
        max_obj = CoordinateObject()

        for r in range(0, self.N):
            for c in range(0, self.N):
                if temp[r][c] == cell:
                    for i in range(len(dx_list)):
                        obj = CoordinateObject()
                        obj.row = r
                        obj.col = c
                        obj.delta_row = dy_list[i]
                        obj.delta_col = dx_list[i]

                        if not self.is_valid_range(obj.next_row(), obj.next_col()):
                            continue

                        if board[obj.next_row()][obj.next_col()] != Cell.EMPTY:
                            continue

                        self.apply_change_to_board(temp, obj, cell)

                        # 재귀적으로 휴리스틱
                        if level > 1:
                            recur_obj = self.__heuristic_impl(temp, cell.get_opposite(), level - 1)
                            # 모든 말이 이미 다 죽었을 수 있음
                            if recur_obj.is_valid():
                                self.apply_change_to_board(temp, recur_obj, cell.get_opposite())

                        diff = self.__get_maximum_diff(temp, cell)

                        if diff > max_diff:
                            max_diff = diff
                            max_obj = obj

                        # reset
                        temp = Utils.copy_board(board)

        return max_obj

    def __get_maximum_diff(self, board, cell):
        player_a_cnt = 0
        player_b_cnt = 0

        for i in range(0, self.N):
            for j in range(0, self.N):
                if board[i][j] == Cell.PLAYER_A:
                    player_a_cnt += 1
                elif board[i][j] == Cell.PLAYER_B:
                    player_b_cnt += 1

        if cell == Cell.PLAYER_A:
            return player_a_cnt - player_b_cnt
        elif cell == Cell.PLAYER_B:
            return player_b_cnt - player_a_cnt
        else:
            raise Exception('only user and computer can use this method')

    def print_board(self, board):
        print('-' * 10)
        print(' ' * 2, end='')
        print(*range(self.N))

        for i, row in enumerate(board):
            print(i, end='|')
            for col in row:
                print(col.get_mark(), end='|')
            print('')

        print('-' * 10)

    def get_winner(self, board):
        player_a_cnt = 0
        player_b_cnt = 0

        for i in range(0, self.N):
            for j in range(0, self.N):
                if board[i][j] == Cell.PLAYER_A: player_a_cnt += 1
                if board[i][j] == Cell.PLAYER_B: player_b_cnt += 1

        if player_a_cnt == 0:
            return Cell.PLAYER_B

        if player_b_cnt == 0:
            return Cell.PLAYER_A

        # 모든 칸이 꽉 찬 경우
        is_board_full = player_a_cnt + player_b_cnt == self.N * self.N
        if is_board_full:
            return Cell.PLAYER_A if player_a_cnt > player_b_cnt else Cell.PLAYER_B

        return Cell.EMPTY


class Ataxx:
    def __init__(self, n):
        self.N = n
        self.board = [[Cell.EMPTY for _ in range(self.N)] for _ in range(self.N)]

        self.__bootstrap()

    def start_game_between_user_and_computer(self):
        print('컴퓨터가 몇 수 앞을 내다보게할까요? 자연수를 입력해주세요. ex) 1, 2, 3...')
        level = int(input())
        service = AtaxxService(self.N, level)

        print('ATAXX 게임을 시작합니다.')
        print('A는 당신의 돌, B는 컴퓨터의 돌입니다.')

        service.print_board(self.board)
        while True:
            print('당신의 차례입니다.')
            obj_user = self.__get_user_input(service)

            service.apply_change_to_board(self.board, obj_user, Cell.PLAYER_A)
            service.print_board(self.board)
            winner = service.get_winner(self.board)
            if winner != Cell.EMPTY:
                print(f'게임이 끝났습니다. {"당신" if winner == Cell.PLAYER_A else "컴퓨터"}이(가) 승리했습니다!')
                break

            print('컴퓨터의 차례입니다.')
            obj_computer = service.heuristic(self.board, Cell.PLAYER_B)

            service.apply_change_to_board(self.board, obj_computer, Cell.PLAYER_B)
            service.print_board(self.board)
            winner = service.get_winner(self.board)
            if winner != Cell.EMPTY:
                print(f'게임이 끝났습니다. {"당신" if winner == Cell.PLAYER_A else "컴퓨터"}이(가) 승리했습니다!')
                break

    def start_game_between_two_computers(self):
        print('컴퓨터 A와 컴퓨터 B가 게임을 플레이합니다.')

        print('컴퓨터 A가 몇 수 앞을 내다보게 할까요? 자연수를 입력해주세요. ex) 1, 2, 3...')
        level_a = int(input())
        service_a = AtaxxService(self.N, level_a)

        print('컴퓨터 B가 몇 수 앞을 내다보게 할까요? 자연수를 입력해주세요. ex) 1, 2, 3...')
        level_b = int(input())
        service_b = AtaxxService(self.N, level_b)

        print('ATAXX 게임을 시작합니다.')
        print('A는 컴퓨터 A의 돌, B는 컴퓨터 B의 돌입니다.')

        while True:
            print('컴퓨터 A의 차례입니다.')
            obj_a = service_a.heuristic(self.board, Cell.PLAYER_A)
            if obj_a.is_valid():
                service_a.apply_change_to_board(self.board, obj_a, Cell.PLAYER_A)
            service_a.print_board(self.board)
            winner = service_a.get_winner(self.board)
            if winner != Cell.EMPTY:
                print(f'게임이 끝났습니다. {"컴퓨터 A" if winner == Cell.PLAYER_A else "컴퓨터 B"}이(가) 승리했습니다!')
                break

            print('컴퓨터 B의 차례입니다.')
            obj_computer = service_b.heuristic(self.board, Cell.PLAYER_B)
            if obj_computer.is_valid():
                service_b.apply_change_to_board(self.board, obj_computer, Cell.PLAYER_B)
            service_b.print_board(self.board)
            winner = service_b.get_winner(self.board)
            if winner != Cell.EMPTY:
                print(f'게임이 끝났습니다. {"컴퓨터 A" if winner == Cell.PLAYER_A else "컴퓨터 B"}이(가) 승리했습니다!')
                break

    def __bootstrap(self):
        self.board[0][0] = Cell.PLAYER_A
        self.board[-1][-1] = Cell.PLAYER_A

        self.board[0][-1] = Cell.PLAYER_B
        self.board[-1][0] = Cell.PLAYER_B

        text = inspect.cleandoc("""
        1. 유저 vs 컴퓨터
        2. 컴퓨터 vs 컴퓨터
        몇 번을 고르시겠습니까? 1, 2 둘 중 하나를 입력하세요.
        """)
        print(text)

        game_type = int(input())
        if game_type == 1:
            self.start_game_between_user_and_computer()
        elif game_type == 2:
            self.start_game_between_two_computers()
        else:
            raise Exception('invalid input')

    def __get_user_input(self, service):
        obj = CoordinateObject()

        while obj.is_row_and_col_not_defined():
            print('이동할 돌을 선택해주세요. (row, col) 순으로 입력해주세요. ex) 2 0')
            row_temp, col_temp = map(int, input().split())

            if not service.is_valid_range(row_temp, col_temp):
                print('보드판의 범위를 넘어갑니다. 다시 입력해주세요.')
                continue

            if self.board[row_temp][col_temp] != Cell.PLAYER_A:
                print('해당 칸에는 유저의 돌이 없습니다. 다시 입력해주세요.')
                continue

            obj.row = row_temp
            obj.col = col_temp

        while obj.is_delta_row_and_delta_col_not_defined():
            print('해당 돌을 얼마만큼 이동할까요? (row, col) 순으로 입력해주세요. ex) row 기준 -2, col 기준 +1만큼 움직이고 싶다면 `-2 1`를 입력하세요. ')
            delta_row_temp, delta_col_temp = map(int, input().split())

            next_row = obj.row + delta_row_temp
            next_col = obj.col + delta_col_temp

            if delta_row_temp > 2 or delta_col_temp > 2:
                print('이동 범위는 두 칸을 넘어갈 수 없습니다.')
                continue

            if not service.is_valid_range(next_row, next_col):
                print('보드판의 범위를 넘어갑니다. 다시 입력해주세요.')
                continue

            if self.board[next_row][next_col] != Cell.EMPTY:
                print('해당 칸에는 이미 돌이 있습니다.')
                continue

            obj.delta_row = delta_row_temp
            obj.delta_col = delta_col_temp

        return obj


if __name__ == '__main__':
    print('판의 크기를 입력해주세요. 홀수만 가능합니다.')
    size = int(input())

    ataxx = Ataxx(size)
